In [22]:
labels = {'pos': 1, 'neg': 0}

df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = '../data/aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as f:
                txt = f.read()
                df = df.append([[txt, labels[l]]], ignore_index=True)
            #break
df.columns = ['review', 'sentiment']
df.head()

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [23]:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('../data/movie.csv', index=False)

In [24]:
df = pd.read_csv('../data/movie.csv')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


# S8.2

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'Th sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
print(count.get_feature_names())
print(count.vocabulary_)
print(count.get_stop_words())
print(bag.toarray())

['and', 'is', 'one', 'shining', 'sun', 'sweet', 'th', 'the', 'two', 'weather']
{'is': 1, 'sun': 4, 'one': 2, 'th': 6, 'the': 7, 'weather': 9, 'shining': 3, 'two': 8, 'and': 0, 'sweet': 5}
None
[[0 1 0 1 1 0 0 1 0 0]
 [0 1 0 0 0 1 0 1 0 1]
 [2 3 2 1 1 1 1 1 1 1]]


In [13]:
count = CountVectorizer(ngram_range=(2, 2))
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'Th sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
print(count.get_feature_names())
print(count.vocabulary_)
print(count.get_stop_words())
print(bag.toarray())

['and one', 'is shining', 'is sweet', 'is two', 'one and', 'one is', 'shining the', 'sun is', 'sweet and', 'th sun', 'the sun', 'the weather', 'weather is']
{'the weather': 11, 'is two': 3, 'the sun': 10, 'sun is': 7, 'one and': 4, 'sweet and': 8, 'weather is': 12, 'is sweet': 2, 'shining the': 6, 'and one': 0, 'is shining': 1, 'th sun': 9, 'one is': 5}
None
[[0 1 0 0 0 0 0 1 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 1 1]
 [2 1 1 1 1 1 1 1 1 1 0 1 1]]


In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)

bag = count.fit_transform(docs)
print(count.get_feature_names())
print(bag.toarray())
print(tfidf.fit_transform(bag).toarray())

['and', 'is', 'one', 'shining', 'sun', 'sweet', 'th', 'the', 'two', 'weather']
[[0 1 0 1 1 0 0 1 0 0]
 [0 1 0 0 0 1 0 1 0 1]
 [2 3 2 1 1 1 1 1 1 1]]
[[ 0.    0.43  0.    0.56  0.56  0.    0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.    0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.25  0.15  0.25  0.19]]


## 8.2.3 data clensing

In [42]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|P|D)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [43]:
a = df.loc[0, 'review'][-50: ]
print(a)
print(preprocessor(a))
print(preprocessor("""</a>This :) is :( test a :-)!"""))

is seven.<br /><br />Title (Brazil): Not Available
is seven title brazil not available
this is test a :) :( :)


In [45]:
df['review'] = df['review'].apply(preprocessor)

In [46]:
df.head()

,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0


## 8.2.4 tokenize

In [48]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(w) for w in text.split()]

tokenizer_porter("runners like running and thus they run")

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [49]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katsuta/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [51]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

[w for w in tokenizer_porter("runners like running and thus they run")
 if w not in stop]

['runner', 'like', 'run', 'thu', 'run']

## 8.2.5 logistic regression

In [52]:
df.head()

,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0


In [54]:
X_train = df.loc[:25000, 'review'].values
X_test = df.loc[25000:, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
y_test = df.loc[25000:, 'sentiment'].values

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1., 10., 100.]}
              ]
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=1,
                           n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 10.9min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', '...11ea0>], 'clf__C': [1.0, 10.0, 100.0], 'vect__ngram_range': [(1, 1)], 'clf__penalty': ['l1', 'l2']}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1

In [57]:
print('Best parameter set: %s' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 10.0, 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x125b11ea0>, 'vect__ngram_range': (1, 1), 'clf__penalty': 'l2'}


In [60]:
clf = gs_lr_tfidf.best_estimator_
print('train %.3f' % gs_lr_tfidf.best_score_)
print('test %.3f' % clf.score(X_test, y_test))


train 0.892
test 0.894


In [73]:
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip the header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [65]:
next(stream_docs(path='../data/movie.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [77]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [78]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='../data/movie.csv')

In [79]:
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)

In [80]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Acc %.3f' % clf.score(X_test, y_test))

Acc 0.867
